In [2]:
from sagemaker import get_execution_role
from sagemaker import Session
from config import executionRole

role = executionRole
sess = Session()
bucket = sess.default_bucket()

In [3]:
# reference: https://github.com/keras-team/keras/issues/14265
!pip install "h5py==2.10.0"
import h5py
import numpy as np
import tensorflow as tf

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.



In [4]:
print(tf.__version__)  # This notebook runs on TensorFlow 1.15.x or earlier
tf_framework_version = tf.__version__

1.15.5


In [5]:
from tensorflow.keras import models


In [7]:
#Convert TensorFlow model to an Amazon SageMaker-readable format

model_path = 'keras_model/nn_model/'
!saved_model_cli show --all --dir {model_path}



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dense_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: serving_default_dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_5'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
 

In [10]:
# Tar the entire directory and upload to Amazon S3.
import tarfile
model_archive = 'model.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True) 

# upload model artifacts to S3
model_data = sess.upload_data(path=model_archive, key_prefix='model')

In [11]:
model_data

's3://sagemaker-ca-central-1-049049517134/model/model.tar.gz'

In [12]:
# deploy the model

from sagemaker.tensorflow.serving import Model
instance_type = 'ml.m4.xlarge' 

sm_model = Model(model_data=model_data, 
framework_version=tf_framework_version,role=role)


The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [13]:
%%time
uncompiled_predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type) 

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!CPU times: user 151 ms, sys: 17.6 ms, total: 168 ms
Wall time: 2min 1s


In [15]:
# The sample model expects an input of shape [1,10]
#data =  [0.35397701,  0.72085963, -0.55290383,  1.2567573 , -0.30380709,0.35397701,  0.72085963, -0.55290383,  1.2567573 , -0.30380709]
input = {
        "instances":[[0.4262894 ,  0.72249761, -0.37445294, -0.31121212,  0.03189628,-0.51861331, -0.39628378, -0.30773688, -0.23297798,  0.49571328],
                 [0.02835102,  0.72249761, -0.22602056,  0.38498714,  1.31851038,-0.51861331, -0.39628378, -0.30773688, -0.23297798,  0.49571328]]
        }
uncompiled_predictor.predict(input)

{'predictions': [[0.320925295], [0.503061771]]}

In [ ]:
#uncompiled_predictor.delete_endpoint()